In [1]:
import pandas as pd

destinations = pd.read_csv("./kaggle/expedia/data/destinations.csv")
test = pd.read_csv("./kaggle/expedia/data/test.csv")
train = pd.read_csv("./kaggle/expedia/data/train.csv")

In [2]:
train["date_time"] = pd.to_datetime(train["date_time"])
train["year"] = train["date_time"].dt.year
train["month"] = train["date_time"].dt.month

In [3]:
def make_key(items):
    return "_".join([str(i) for i in items])

match_cols = ["srch_destination_id"]
cluster_cols = match_cols + ['hotel_cluster']
groups = train.groupby(cluster_cols)
top_clusters = {}
for name, group in groups:
    clicks = len(group.is_booking[group.is_booking == False])
    bookings = len(group.is_booking[group.is_booking == True])
    
    score = bookings + .15 * clicks
    
    clus_name = make_key(name[:len(match_cols)])
    if clus_name not in top_clusters:
        top_clusters[clus_name] = {}
    top_clusters[clus_name][name[-1]] = score

In [4]:
most_common_clusters = list(train.hotel_cluster.value_counts().head().index)
import operator

cluster_dict = {}
for n in top_clusters:
    tc = top_clusters[n]
    top = [l[0] for l in sorted(tc.items(), key=operator.itemgetter(1), reverse=True)[:5]]
    cluster_dict[n] = top

In [5]:
import ml_metrics as metrics
preds = []
for index, row in test.iterrows():
    key = make_key([row[m] for m in match_cols])
    if key in cluster_dict:
        preds.append(cluster_dict[key])
    else:
        preds.append([])
#metrics.mapk([[l] for l in test["hotel_cluster"]], preds, k=5)



In [6]:
match_cols = ['user_location_country', 'user_location_region', 'user_location_city', 'hotel_market', 'orig_destination_distance']

groups = train.groupby(match_cols)
    
def generate_exact_matches(row, match_cols):
    index = tuple([row[t] for t in match_cols])
    try:
        group = groups.get_group(index)
    except Exception:
        return []
    clus = list(set(group.hotel_cluster))
    return clus
exact_matches = []
for i in range(test.shape[0]):
    exact_matches.append(generate_exact_matches(test.iloc[i], match_cols))


ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-ca3bf3ca2f41>", line 15, in <module>
    exact_matches.append(generate_exact_matches(test.iloc[i], match_cols))
  File "<ipython-input-6-ca3bf3ca2f41>", line 8, in generate_exact_matches
    group = groups.get_group(index)
  File "C:\Anaconda3\lib\site-packages\pandas\core\groupby.py", line 650, in get_group
    return obj.take(inds, axis=self.axis, convert=False)
  File "C:\Anaconda3\lib\site-packages\pandas\core\generic.py", line 1371, in take
    convert=True, verify=True)
  File "C:\Anaconda3\lib\site-packages\pandas\core\internals.py", line 3628, in take
    axis=axis, allow_dups=True)
  File "C:\Anaconda3\lib\site-packages\pandas\core\internals.py", line 3515, in reindex_indexer
    for blk in self.blocks]
  File "C:\Anaconda3\lib\site-packages\pandas\core\internals.py", l


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable

In [ ]:
def f5(seq, idfun=None): 
    if idfun is None:
        def idfun(x): return x
    seen = {}
    result = []
    for item in seq:
        marker = idfun(item)
        if marker in seen: continue
        seen[marker] = 1
        result.append(item)
    return result
    
full_preds = [f5(exact_matches[p] + preds[p] + most_common_clusters)[:5] for p in range(len(preds))]
mapk([[l] for l in test["hotel_cluster"]], full_preds, k=5)

In [ ]:
write_p = [" ".join([str(l) for l in p]) for p in full_preds]
write_frame = ["{0},{1}".format(test["id"][i], write_p[i]) for i in range(len(full_preds))]
write_frame = ["id,hotel_clusters"] + write_frame
with open("predictions.csv", "w+") as f:
    f.write("\n".join(write_frame))

In [24]:
list(set(group.hotel_cluster))

[76]

In [26]:
generate_exact_matches(test.iloc[2], match_cols)

[91]

In [30]:
g=0
exact_matches = []
for i in range(test.shape[0]):
    exact_matches.append(generate_exact_matches(test.iloc[i], match_cols))
    g=g+1
    if g==100:
        print(i)
        g=0


99
199
299
399
499
599
699
799
899
999
1099
1199
1299
1399
1499
1599
1699


KeyboardInterrupt: 

In [31]:
a=pd.DataFrame(groups).rename(columns={0:'hotel_cluster'})

MemoryError: 

In [8]:
len(exact_matches)

19421